#### jieba构建词袋模型

- 首先，引入jieba分词库、语料、停用词(标点符号集合，可以手动添加或用一个文本字典代替)

In [1]:
import jieba

# 定义停用词
stopwords = ["，", "。", "：", "；", "？", "！"]

# 定义语料
content = ["机器学习带动人工智能飞速的发展。", "深度学习带动人工智能飞速的发展。", "机器学习和深度学习带动人工智能飞速的发展！"]


In [2]:
# 对语料进行分词操作，用来lcut()方法
segs_1 = [jieba.lcut(con) for con in content]
segs_1

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.679 seconds.
Prefix dict has been built succesfully.


[['机器', '学习', '带动', '人工智能', '飞速', '的', '发展', '。'],
 ['深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '。'],
 ['机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '！']]

In [3]:
# 因为中文语料中带有停用词和标点符号，所以需要去停用词和标点符号。这里的语料很小，所以只去掉标点符号。

tokenized = []
for sentence in segs_1:
    words = []
    for word in sentence:
        if word not in stopwords:
            words.append(word)
    tokenized.append(words)
print(tokenized)  # 去掉停用词后的结果

[['机器', '学习', '带动', '人工智能', '飞速', '的', '发展'], ['深度', '学习', '带动', '人工智能', '飞速', '的', '发展'], ['机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展']]


In [4]:
# 将所有的分词结果放入一个list中，即：取并集、再去重、获取对应的特征词。

bag_of_words = [x for item in segs_1 for x in item if x not in stopwords]

bag_of_words = list(set(bag_of_words))
bag_of_words

['深度', '带动', '和', '飞速', '的', '机器', '发展', '学习', '人工智能']

In [5]:
bag_of_word2vec = []
for sentence in tokenized:
    tokens = [1 if token in sentence else 0 for token in bag_of_words]
    bag_of_word2vec.append(tokens)
bag_of_word2vec  # 词袋向量

[[0, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1]]

#### gensim构建词袋模型

In [8]:
from gensim import corpora
import gensim

dictionary = corpora.Dictionary(tokenized)

# 保存词袋典
dictionary.save("deerwester.dict")
print(dictionary)  # 得到的结果不全

Dictionary(9 unique tokens: ['深度', '飞速', '的', '和', '发展']...)


d:\python35\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
# 查看词典和下标id的映射
print(dictionary.token2id)

{'深度': 7, '飞速': 6, '的': 5, '和': 8, '发展': 1, '学习': 2, '带动': 3, '人工智能': 0, '机器': 4}


In [11]:
corpus = [dictionary.doc2bow(sentence) for sentence in segs_1]  
# doc2bow()函数计算每个不同单词的出现次数，将单词转换为整数单词id并将结果作为稀疏向量返回
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


#### 词向量模型

In [6]:
from gensim.models import Word2Vec
import jieba

# 停用词、标点符号

stopwords = ["，", "。", "：", "；", ".", "`", "、", "/", "？", "-", "+", "*", "&", "(", ")", '“', '”' ]

sentences = [
    "长江是中国第一大河，干流全长6397公里（以沱沱河为源），一般称6300公里。流域总面积一百八十余万平方公里，年平均入海水量约九千六百余亿立方米。以干流长度和入海水量论，长江均居世界第三位。",
    "黄河，中国古代也称河，发源于中华人民共和国青海省巴颜喀拉山脉，流经青海、四川、甘肃、宁夏、内蒙古、陕西、山西、河南、山东9个省区，最后于山东省东营垦利县注入渤海。干流河道全长5464千米，仅次于长江，为中国第二长河。黄河还是世界第五长河。",
    "黄河,是中华民族的母亲河。作为中华文明的发祥地,维系炎黄子孙的血脉.是中华民族民族精神与民族情感的象征。",
    "黄河被称为中华文明的母亲河。公元前2000多年华夏族在黄河领域的中原地区形成、繁衍。",
    "在兰州的“黄河第一桥”内蒙古托克托县河口镇以上的黄河河段为黄河上游。",
    "黄河上游根据河道特性的不同，又可分为河源段、峡谷段和冲积平原三部分。",
    "黄河,是中华民族的母亲河。"
]

# 分词后去掉标点符号
sentences = [jieba.lcut(sen) for sen in sentences]
print(sentences)
print("----------------------------------------------------------------分割线----------------------------------------------------------------")


tokenized = []
for sentence in sentences:
    words = []
    for word in sentence:
        if word not in stopwords:
            words.append(word)
    tokenized.append(words)
print(tokenized)

[['长江', '是', '中国', '第一', '大河', '，', '干流', '全长', '6397', '公里', '（', '以', '沱沱河', '为源', '）', '，', '一般', '称', '6300', '公里', '。', '流域', '总面积', '一百八十', '余万平方公里', '，', '年', '平均', '入海', '水量', '约', '九千', '六百余', '亿立方米', '。', '以', '干流', '长度', '和', '入海', '水量', '论', '，', '长江', '均', '居', '世界', '第三位', '。'], ['黄河', '，', '中国', '古代', '也', '称河', '，', '发源', '于', '中华人民共和国', '青海省', '巴颜喀拉山', '脉', '，', '流经', '青海', '、', '四川', '、', '甘肃', '、', '宁夏', '、', '内蒙古', '、', '陕西', '、', '山西', '、', '河南', '、', '山东', '9', '个', '省区', '，', '最后', '于', '山东省', '东营', '垦利县', '注入', '渤海', '。', '干流', '河道', '全长', '5464', '千米', '，', '仅次于', '长江', '，', '为', '中国', '第二', '长河', '。', '黄河', '还是', '世界', '第五', '长河', '。'], ['黄河', ',', '是', '中华民族', '的', '母亲河', '。', '作为', '中华文明', '的', '发祥地', ',', '维系', '炎黄子孙', '的', '血脉', '.', '是', '中华民族', '民族', '精神', '与', '民族', '情感', '的', '象征', '。'], ['黄河', '被', '称为', '中华文明', '的', '母亲河', '。', '公元前', '2000', '多年', '华夏', '族', '在', '黄河', '领域', '的', '中原地区', '形成', '、', '繁衍', '。'], ['在', '兰州', '的', '“', '黄河', '第一', '桥', 

In [10]:
# 训练模型
model = Word2Vec(tokenized, sg=1, size=100, window=5, min_count=2, negative=1, sample=0.001, hs=1, workers=4)

# sg=1表示是skip-gram算法，对低频词敏感，默认sg=0为CBOW算法
# size是输出词向量的维度，值太小会导致词映射因为冲突而影响结果，值太大则耗内存使得算法计算缓慢，一般值取100到200之间
# window是句子当前词与目标词之间的最大距离
# min_count是对词进行过滤，频率小于min_count的单词会自动忽略，默认是5
# negative和sample可根据训练结果进行微调，sample表示更高频率的词被随机下采样到所设置的阈值，默认是1e-3
# hs表示分层softmax将会被使用，默认hs=0且negative不为0，则负采样将会被使用


In [12]:
 print(model.similarity("黄河", "黄河"))

1.0


d:\python35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [14]:
 print(model.similarity("黄河", "长江"))

-0.082179666


d:\python35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


In [15]:
print(model.most_similar(positive=["黄河", "母亲河"], negative=["长江"]))

[('入海', 0.2101619988679886), ('内蒙古', 0.14537617564201355), ('公里', 0.12905599176883698), ('是', 0.07895912230014801), ('和', 0.06504575908184052), ('的', 0.059060487896203995), ('干流', 0.055452100932598114), ('全长', 0.05235753208398819), ('民族', 0.04623307287693024), ('段', 0.038957379758358)]


d:\python35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
